<a href="https://colab.research.google.com/github/thisissamuca/GOES_16/blob/main/FDCF_POINTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install rioxarray xarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 24.5 MB/s eta 0:00:00


In [4]:
import os
from glob import glob
from pathlib import Path

from datetime import datetime

import xarray as xr
import rioxarray as rxr
from rioxarray.raster_dataset import RasterDataset

import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
basePath = Path('/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/ABI-L2-FDCF/netCDF')


In [6]:
basePath = Path('/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/ABI-L2-FDCF/netCDF')

# Abre um arquivo NetCDF de exemplo para teste
ds = xr.open_dataset(basePath)
ds['Temp']  # Acessa a variável de temperatura

<xarray.DataArray 'Temp' (y: 5424, x: 5424)> Size: 118MB
[29419776 values with dtype=float32]
Coordinates:
    t                   datetime64[ns] 8B ...
  * y                   (y) float32 22kB 0.1518 0.1518 ... -0.1518 -0.1518
  * x                   (x) float32 22kB -0.1518 -0.1518 ... 0.1518 0.1518
    y_image             float32 4B ...
    x_image             float32 4B ...
    sunglint_angle      float32 4B ...
    local_zenith_angle  float32 4B ...
    solar_zenith_angle  float32 4B ...
Attributes:
    long_name:            ABI L2+ Fire-Hot Spot Characterization: Fire Temper...
    standard_name:        fire_temperature
    valid_range:          [ 0 -6]
    units:                K
    resolution:           y: 0.000056 rad x: 0.000056 rad
    grid_mapping:         goes_imager_projection
    cell_measures:        area: Area
    cell_methods:         sunglint_angle: point (no pixel produced) local_zen...
    ancillary_variables:  DQF

In [ ]:
#Extensão da região hidrográfica do Rio Paraguai
#xmin, ymin, xmax, ymax = -59.7117080000000016,-22.3014335000000017, -53.1477660999999983,-14.1449678999999993
#xmin, ymin, xmax, ymax = 1526628,-2354717, 2259261,-1517531
ixmin, iymin, ixmax, iymax = 3472, 3470, 3838, 3887

def getNomeArquivo(ds:xr.Dataset):

    tInicio = tFim = datetime.fromisoformat(str(ds['t'].values[0])).strftime('%Y%j%H%M%S')
    tFim = datetime.fromisoformat(str(ds['t'].values[-1])).strftime('%Y%j%H%M%S')
    tProcessamento = datetime.now().strftime('%Y%j%H%M%S')

    nomeArquivo = f'OR_ABI-L2-FDCF-M6_G16_s{tInicio}_e{tFim}_c{tProcessamento}.nc'
    return nomeArquivo

basePath = Path('../../dados/goes-16/fdcf/')

In [ ]:
ds = xr.open_dataset(list(basePath.glob('2020/jun/*/*.nc'))[0])
ds['Temp']

In [24]:
basePath = Path('/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/ABI-L2-FDCF/netCDF')
file_list = list(basePath.glob('*.nc'))
file_list[0]

PosixPath('/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/ABI-L2-FDCF/netCDF/OR_ABI-L2-FDCF-M6_G16_s20201551340172_e20201551349480_c20201551350063.nc')

In [25]:
basePath = Path('/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/ABI-L2-FDCF/netCDF')

# Find all netCDF files for the year 2020
file_list = list(basePath.glob('*.nc'))

# Check if any files were found
if file_list:
    # Abre um arquivo NetCDF de exemplo para teste
    ds = xr.open_dataset(file_list[0])
    print(ds)
else:
    print("No NetCDF files found for the year 2020.")

<xarray.Dataset> Size: 588MB
Dimensions:                                           (y: 5424, x: 5424,
                                                       number_of_time_bounds: 2,
                                                       number_of_image_bounds: 2,
                                                       number_of_sunglint_angle_bounds: 2,
                                                       number_of_LZA_bounds: 2,
                                                       number_of_SZA_bounds: 2)
Coordinates:
    t                                                 datetime64[ns] 8B ...
  * y                                                 (y) float32 22kB 0.1518...
  * x                                                 (x) float32 22kB -0.151...
    y_image                                           float32 4B ...
    x_image                                           float32 4B ...
    sunglint_angle                                    float32 4B ...
    local_zenith_angle    

In [ ]:
basePath = Path('/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/ABI-L2-FDCF/netCDF')



executar = True  # Flag para controlar execução

if(executar):
    append = True  # Controla se deve anexar ou criar novo arquivo

    # Processa arquivos de 2024
    for periodo in list(basePath.glob('*.nc'))[:]:
        periodo = Path(periodo)

        with rxr.open_rasterio(periodo, band_as_variable=False) as ds:
            # Pré-processamento dos dados
            ds = ds.squeeze('band')  # Remove dimensão de banda se existir
            #ds = ds.isel(x=slice(ixmin, ixmax), y=slice(iymin, iymax))  # Recorta para ROI
            ds = ds.rio.reproject(dst_crs=f'EPSG:4674')  # Reprojeta para SIRGAS 2000
            ds = ds[['Area', 'Temp', 'Mask', 'Power', 'DQF']]  # Seleciona variáveis

            # Converte para DataFrame
            df = ds.to_dataframe(['x', 'y']).reset_index(drop=False)
            df['time'] = pd.to_datetime(ds.attrs['time_coverage_start'])
            # Renomeia colunas
            df = df.rename(columns={'x':'lon', 'y':'lat', 't':'time', 'Area':'Area_m2', 'Temp': 'Temp_K'})
            df = df.set_index(['lon', 'lat', 'time'])[['Area_m2', 'Temp_K', 'Mask', 'Power', 'DQF']]

            ##Filtro de pixels
            #df = df[((df['Mask'] == 13) | (df['Mask'] == 33) & (df['DQF'] == 0))]
            df = df[(df['Mask'] == 10) & (df['DQF'] == 0)]  # Filtra apenas pixels válidos

            # Define caminho para salvar
            nmArquivoSalvar = Path(basePath, 'focos de incêndio', 'focos_incendio.csv')
            nmArquivoSalvar.parent.mkdir(parents=True, exist_ok=True)

            # Salva em CSV (anexando ou criando novo)
            df.to_csv(nmArquivoSalvar, sep=',', header=not append, mode='a' if append else 'w')

            append=True  # Próximas iterações anexam ao arquivo

In [ ]:
dicTimeZone = {'Cuiabá': 'America/Cuiaba', 'Campo Grande':'Ameriza/Campo_Grande', 'São Paulo':'America/Sao_Paulo'}

In [ ]:
df = pd.read_csv('../../dados/goes-16/fdcf/focos de incêndio/focos_incendio.csv', index_col=(0,1))

idxData = pd.DatetimeIndex(data=df['time'], tz='UTC').tz_convert(dicTimeZone['Cuiabá']).tz_localize(None) + pd.DateOffset(microsecond=0, second=0)
df['ano'] = pd.to_datetime(idxData).year
df['mes'] = pd.to_datetime(idxData).month
df['dia'] = pd.to_datetime(idxData).day
df['hora'] = pd.to_datetime(idxData).hour
df['minuto'] = pd.to_datetime(idxData).minute

df['Temp_C'] = df['Temp_K'] -273.5

df = df.drop(columns=['time', 'Temp_K'])
df = df.set_index(keys=idxData, append=True)
df = df.sort_index()
df = df[['ano', 'mes', 'dia', 'hora', 'Area_m2', 'Power', 'Temp_C']]
df.to_csv('../../dados/goes-16/fdcf/focos de incêndio/focos_incendio_filtro.csv')

In [ ]:
#pd.to_datetime(idxData)
df[df['Temp_C'] >= 0].sort_values('Temp_C', ascending=True).head(60)

In [ ]:
for dia in list(basePath.glob('2024/*/*/'))[:2]:
    dia = Path(dia)

    with xr.open_mfdataset(list(dia.glob('*.nc'))[:], concat_dim='t', combine='nested',
                           parallel=True, engine='h5netcdf') as ds:
        ds = ds.isel(x=slice(ixmin, ixmax), y=slice(iymin, iymax))
        ds = ds[['Area', 'Temp', 'Mask', 'Power', 'DQF', 'goes_imager_projection']]

        pathArquivoSaida = Path(dia.parent, 'composicao_diaria', getNomeArquivo(ds))

        if(not pathArquivoSaida.exists()):
            ds.to_netcdf(path=pathArquivoSaida)
            pathArquivoSaida.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
for mes in list(basePath.glob('2024/*'))[:1]:
    mes = Path(mes)

    with xr.open_mfdataset(list(mes.glob('*/*.nc'))[:], concat_dim='t', combine='nested',
                           parallel=True, engine='h5netcdf') as ds:
        ds = ds.isel(x=slice(ixmin, ixmax), y=slice(iymin, iymax))
        ds = ds[['Area', 'Temp', 'Mask', 'Power', 'DQF', 'goes_imager_projection']]

        pathArquivoSaida = Path(mes.parent, 'composicao_diaria', getNomeArquivo(ds))

        if(not pathArquivoSaida.exists()):
            ds.to_netcdf(path=pathArquivoSaida)
            pathArquivoSaida.parent.mkdir(parents=True, exist_ok=True)